### LSTM ###

In [19]:
import pandas as pd
from sklearn.utils import shuffle

In [20]:
import pandas as pd
import re
import string
from tqdm import tqdm
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

### Data Crawling ###

In [21]:
import pandas as pd
import re
import string
from tqdm import tqdm
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

class DataCleaning:
  # Initialization
  factory     = StemmerFactory()
  stemmer     = factory.create_stemmer()
  kamus_alay2 = pd.read_csv('https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv')
  kamus_alay2 = kamus_alay2.filter(['slang', 'formal'], axis=1)
  kamus_alay2 = kamus_alay2.drop_duplicates(subset=['slang'], keep='first')
  kamus_alay2 = kamus_alay2.set_index('slang')
  stopword1   = list(pd.read_csv('https://raw.githubusercontent.com/datascienceid/stopwords-bahasa-indonesia/master/stopwords_id_satya.txt', header = None)[0])
  custom_word = ['nya', 'tolong', 'iya', 'guna', 'kasih','oemimoin','buka', 'hari', 'sih', 'mohon', 'baru', 'ovo', 'dana', 'linkaja', 'suatau', 'aplikasi', 'Disamoing', 'oemimoin']

  @classmethod
  def CleanDataFrame(cls, df, col_name, jum_minimum=None, minimum_kata=0):

    final_list_clean = []
    final_list_kotor = []

    if jum_minimum == None: jum_minimum = len(df)
    if len(df) < jum_minimum: raise "Jumlah Data Yang Diinginkan melebihi Data yang Ada"
    i = 0
    current = 0
    
    while i < len(df):
      current_kalimat = df.loc[i][col_name]
      clean_kalimat = cls.__cleanSentence__(current_kalimat)
      if (len(clean_kalimat.split(' ')) > minimum_kata):
        final_list_clean.append(clean_kalimat)
        final_list_kotor.append(current_kalimat)
        current += 1
        if current % 10 == 0:
          print("Memproses {} data".format(current))

      if current == jum_minimum:
        break
      i += 1
    
    data = {
        'raw': final_list_kotor,
        'processed': final_list_clean
    }

    return pd.DataFrame(data)

  @classmethod
  def CleanSentence(cls, text):
    return cls.__cleanSentence__(text)

  @classmethod
  def __cleanSentence__(cls, text):
    '''
    Melakukan prapemrosesan pada suatu kalimat dengan menghilangkan formatting pada kalimat,
    menghilangkan stopword pada kalimat, mengganti kata alay yang sudah terdefinisikan, serta
    melakukan stemming kalimat tersebut.
    '''
    print(text)
    # #
    # Cleaning Formatted Text using Regex
    # #
    text = re.sub(r'http\S+', '', text)
    text = re.sub('(@\w+|#\w+)','',text)
    #will replace the html characters with " "
    text=re.sub('<.*?>', '', text)  
    #To remove the punctuations

    ## kuganti jadi gini biar pasti, kalau pakai cara yang dulu, banyak kata2 yang kegabung -kaenova
    temp_text = list(text)
    for i in range(len(temp_text)):
      if temp_text[i] in string.punctuation:
        temp_text[i] = " "
    text = ''.join(temp_text)
    ## sebelumnya kaya gini -kaenova
    # text = text.translate(str.maketrans(' ',' ',string.punctuation))

    #will consider only alphabets
    text = re.sub('[^a-zA-Z]',' ',text) 
    #will replace newline with space
    text = re.sub("\n"," ",text)
    #will convert to lower case
    text = text.lower()
    # will replace a word
    text = re.sub("(username|user|url|rt|xf|fx|xe|xa)\s|\s(user|url|rt|xf|fx|xe|xa)","",text)
    # will repalce repated char
    text = re.sub(r'(\w)(\1{2,})', r"\1", text)
    # will replace single word
    text = re.sub(r"\b[a-zA-Z]\b","",text)
    # will replace space more than one
    text = re.sub('(s{2,})',' ',text)
    # will join the words
    text=' '.join(text.split())

    text_split = text.split(' ')
    # #
    # Mengganti kata-kata yang tidak baku
    # aku gapakai try catch lagi, lebih simple malah ini
    # #
    for i in range(len(text_split)):
      if text_split[i] in cls.kamus_alay2.index:
        text_split[i] = cls.kamus_alay2.loc[text_split[i]]['formal']
      else:
        pass

    # #
    # Stemming
    # #
    stemmed_text = cls.stemmer.stem(text)

    # #
    # Removing Stopwords and custom word
    # #
    temp_text_split = []
    for i in range(len(text_split)):
      if (text_split[i] not in cls.stopword1) and (text_split[i] not in cls.custom_word) and (type(text_split[i]) == str):
        temp_text_split.append(text_split[i])

    final_text = ' '.join(temp_text_split)
    
    return final_text

In [22]:
import os
import pandas as pd

directory = 'C:\\Users\\user\\PENELITIAN SAINS DATA\\BARU\\data\\rawV2'
out_directory = 'C:\\Users\\user\\PENELITIAN SAINS DATA\\BARU\\data\\processedV4'

if directory[len(directory)-1] != '/':
    directory = directory + '/'

if out_directory[len(out_directory)-1] != '/':
    out_directory = out_directory + '/'

files = os.listdir(directory)

for file in files:
    if file[len(file)-4:] == '.csv':
        print(file)
        df = pd.read_csv(directory+file)
        df_temp = DataCleaning.CleanDataFrame(df, 'Comment', None, 10)
        df_temp.to_csv(out_directory+file, index=False)

data1.csv
jika anggaran di total mngenai pembangunan jakarta slama ini adalah ratusan triliun udah bisa bangun ibu kota baru dgn hasil karya anak bangsa yg lbh baik tnpa istana peninggalan belanda
Alhamdulillah....
Dibangun oleh beliau, dan dinikmati oleh generasi berikutnya dan pemimpin2 selanjutnya. Btw saya kira kalau pengganti 5 tahun menjabat, trus pak Jokowi nyapres lagi apa bisa ???
Berarti kemungkinan 2 periode lg kalau cukup umurnya?
Apa gitu ya?
Semua akan di nikmati oleh2 cucu kita .di jaman kita hanya membangun.trimakasi pak Jokowi untuk indonesia yg lebih maju dan menjadi negara adidaya .kita sudh berjanji kepada dunia 50th-100th kedepan indonesia sudh punya wajah negara yg luar biasa
Pak jokowi jangan hanya pembangunan nya saja ..kesejahteraan karyawan tolong pehatikan
Air susah
Bus terlambat 
Gaji banyak potongan 
Tindak pak oknum2 nya
Pak die tolong dong aq mesan mcb kwh listrik dari bulan ramadan belum datang 😢
❤❤❤❤❤❤
Semoga Allah SWT senantiasa memberikan kesehatan da

### Untuk Data Label ###

In [23]:
DataCleaning.CleanSentence('Mau jadi negara maju kok gak mau. Populasi di jakarta sidah 70 juta jiwa lebih. Sedangkan idealnya suatau kota bisa dikatakan baik adalah tidak lebih dari 2 juta jiwa seperti IKN nantinya. Aoa yg diharapkan dr kota yg sudah padat, banjir dan tanahnya turun tiap tahun. Justru dg adanya IKN akan memancing investasi dan kepercayaan asing dan membuat pola kerja baru dg tempat yg lebih baik juga utk pemerataan pembangunan. Selain itu indonesia akan tampil menjadi oemimoin dunia . Disamoing itu ancaman bencana megatrush akan menghantam jakarta.')

Mau jadi negara maju kok gak mau. Populasi di jakarta sidah 70 juta jiwa lebih. Sedangkan idealnya suatau kota bisa dikatakan baik adalah tidak lebih dari 2 juta jiwa seperti IKN nantinya. Aoa yg diharapkan dr kota yg sudah padat, banjir dan tanahnya turun tiap tahun. Justru dg adanya IKN akan memancing investasi dan kepercayaan asing dan membuat pola kerja baru dg tempat yg lebih baik juga utk pemerataan pembangunan. Selain itu indonesia akan tampil menjadi oemimoin dunia . Disamoing itu ancaman bencana megatrush akan menghantam jakarta.


'jadi negara maju populasi jakarta sidah juta jiwa idealnya kota dikatakan baik juta jiwa ikn aoa diharapkan kota padat banjir tanahnya turun tahun ikn memancing investasi kepercayaan asing membuat pola kerja tempat baik untuk pemerataan pembangunan indonesia tampil menjadi dunia disamoing ancaman bencana megatrush menghantam jakarta'

### Data Train Sentiment General External ###

In [24]:
df_train = pd.read_csv('C:\\Users\\user\\PENELITIAN SAINS DATA\\BARU\\data\\trainingEksternalV1\\train.csv')
df_clean = DataCleaning.CleanDataFrame(df_train, 'Text Tweet', None, 10)

Banyak akun kloning seolah2 pendukung #agussilvy mulai menyerang paslon #aniessandi dengan opini dan argumen pmbenaran..jangan terkecoh
#agussilvy bicara apa kasihan yaa...lap itu air matanya wkwkwkwk
Kalau aku sih gak nunggu hasil akhir QC tp lagi nunggu motif cuitan pak @SBYudhoyono kayak apa.. pasca #AgusSilvy Nyungsep..
Kasian oh kasian dengan peluru 1milyar untuk tiap RW #agussilvy tidak mempan menangin pilkada #QuickCount #PilkadaSerentak2017
Maaf ya pendukung #AgusSilvy..hayo dukung #AniesSandi diputaran 2 @ronavioleta @NetizenTofa
aneh deh lebay masa ini di sangkut pautkan sama kandidat Calgub, anda lebay seperti yg anda dukung #agussilvy
Kepada Allah SWT kami ucapkan rasa syukur dan kepada pak @SBYudhoyono @presidenSBY #AHY kami sampaikan terimakasih. Makna 17% suara anda.
Terima Kasih teruntuk pendukung #AHY #SYLVI kalianlah pembeda kali ini
Trima Kasih atas KeIstiqomahan Relawan #AHY @AgusYudhoyono lah yg tlh mengantarkan #AniesSandiDKI1 #JakartaGubernurBaru dg cukup signifi

In [25]:
df_clean

,raw,processed
0,Kalau aku sih gak nunggu hasil akhir QC tp lag...,menunggu hasil akhir qc menunggu motif cuitan ...
1,ya pastilah #FPI kan ga suka ama Ahok jd pas #...,ya suka ahok jadi pas kalah otomatis dukungann...
2,"#ahy, pakdhe karwo, Pilgub Jatim, Pak Dhe Karw...",pakdhe karwo pilgub jatim pak deh karwo isyara...
3,"Di manakah #AHY... Sang Ksatria, Samurai, Praj...",manakah sang ksatria samurai prajurit pembangu...
4,"Kini, ketika #AHY berada di luar arena, tabir ...",berada luar arena tabir kegelapan tiada harapa...
...,...,...
602,Film Kartini *sangat niat banget lihat film in...,film kartini niat banget lihat film keren dima...
603,KARTINI adalah salah satu film Indonesia berte...,kartini salah satu film indonesia bertema seja...
604,@filmziarah film yang tenang dan menghanyutkan...,film tenang menghanyutkan salut mbah ponco mem...
605,Film yg amat menarik. Kisah cinta & kesetiaan ...,film menarik kisah cinta kesetiaan disajikan s...


In [26]:
df_clean_temp = df_clean.copy()
df_clean_temp

,raw,processed
0,Kalau aku sih gak nunggu hasil akhir QC tp lag...,menunggu hasil akhir qc menunggu motif cuitan ...
1,ya pastilah #FPI kan ga suka ama Ahok jd pas #...,ya suka ahok jadi pas kalah otomatis dukungann...
2,"#ahy, pakdhe karwo, Pilgub Jatim, Pak Dhe Karw...",pakdhe karwo pilgub jatim pak deh karwo isyara...
3,"Di manakah #AHY... Sang Ksatria, Samurai, Praj...",manakah sang ksatria samurai prajurit pembangu...
4,"Kini, ketika #AHY berada di luar arena, tabir ...",berada luar arena tabir kegelapan tiada harapa...
...,...,...
602,Film Kartini *sangat niat banget lihat film in...,film kartini niat banget lihat film keren dima...
603,KARTINI adalah salah satu film Indonesia berte...,kartini salah satu film indonesia bertema seja...
604,@filmziarah film yang tenang dan menghanyutkan...,film tenang menghanyutkan salut mbah ponco mem...
605,Film yg amat menarik. Kisah cinta & kesetiaan ...,film menarik kisah cinta kesetiaan disajikan s...


In [27]:
df_train = pd.read_csv('C:\\Users\\user\\PENELITIAN SAINS DATA\\BARU\\data\\trainingEksternalV1\\train.csv')
df_train = df_train.rename(columns={'Text Tweet':'raw'})
df_train

,Sentiment,raw
0,-1,Banyak akun kloning seolah2 pendukung #agussil...
1,-1,#agussilvy bicara apa kasihan yaa...lap itu ai...
2,-1,Kalau aku sih gak nunggu hasil akhir QC tp lag...
3,-1,Kasian oh kasian dengan peluru 1milyar untuk t...
4,-1,Maaf ya pendukung #AgusSilvy..hayo dukung #Ani...
...,...,...
1895,1,Fargo juga adaptasi dari film yang cukup berha...
1896,1,637.000 waw ini sangat keren flm horor dng jum...
1897,1,@filmziarah film yang tenang dan menghanyutkan...
1898,1,Film yg amat menarik. Kisah cinta & kesetiaan ...


In [28]:
from sklearn.utils import shuffle

final_df = pd.merge(df_train, df_clean_temp, how='inner', on='raw')
final_df = final_df.drop_duplicates(['raw'])
final_df = final_df.reset_index(drop=True)

final_df_negative = final_df[final_df['Sentiment'] == -1]
final_df_positive = final_df[final_df['Sentiment'] == 1]

if len(final_df_negative) < len(final_df_positive):
    final_df_positive = final_df_positive.head(len(final_df_negative))
else:
    final_df_negative = final_df_negative.head(len(final_df_positive))

print(len(final_df_negative), len(final_df_positive))

final_banget_df = pd.concat([final_df_negative, final_df_positive])
final_banget_df = shuffle(final_banget_df)
final_banget_df.to_csv("C:\\Users\\user\\PENELITIAN SAINS DATA\\BARU\\data\\trainingEksternalV1\\train_cleaned.csv", index=False)
     

274 274
